# Getting Quality Predictions for World Politicians' Wikipedia Articles

### Homework #2 – Data 512
### Daniel Vogler

# Combining Politician and Population Datasets

## Environment Setup

In [1]:
import pandas as pd
import json

## Data Import

In [7]:
# get politicians with predicted article quality...

politician_article_quality_filepath = "../output_data/quality_predictions.json"

with open(politician_article_quality_filepath, "r") as f:
    politician_article_quality_js = json.load(f)

politician_article_quality_df = pd.DataFrame(politician_article_quality_js)

print(len(politician_article_quality_df))

7155
